In [1]:
import numpy as np

import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.utils import to_categorical

In [2]:
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:

path = "/content/drive/My Drive/dataset/_basic"
pokemon_types = os.listdir(path) 
print("Total number of pokemon types: " + str(len(pokemon_types)))

Total number of pokemon types: 57


In [10]:
image_set = {}
for c in pokemon_types:
    if not c.startswith("."):
        image_set[c] = len(os.listdir(os.path.join(path, c)))
    
print("Total number of images in dataset: " + str(sum(list(image_set.values()))))


Total number of images in dataset: 4371


In [11]:

labels = [i[0] for i in image_set.items()]


In [12]:
X = [] 
Y = [] 


for i in pokemon_types:
    
    dir_path = os.path.join(path, i)
    label = labels.index(i) 
        

    for j in os.listdir(dir_path):
        image = cv2.imread(os.path.join(dir_path, j))
            
        try:
            smaller = cv2.resize(image, (96, 96)) 
            X.append(smaller)
            Y.append(label)
            
            
        except:
            print(os.path.join(dir_path, j), "Error")
            continue       
            
print("completed")

/content/drive/My Drive/dataset/_basic/Caterpie/dcd36168d2bd470aaca333fb3bdf6bfa.svg Error
/content/drive/My Drive/dataset/_basic/Growlithe/f7a87c057dc94a32a0d5aedd1abed088.svg Error
/content/drive/My Drive/dataset/_basic/Venonat/56425d5f66e64584afbfc797b93c7ee5.ico Error
/content/drive/My Drive/dataset/_basic/Scyther/a8734f10941a4deb95fcf7388e4fe8e2.svg Error
/content/drive/My Drive/dataset/_basic/Voltorb/94f443d834f94c1e8e6b088d879646d4.gif~c200 Error
/content/drive/My Drive/dataset/_basic/Machamp/546d68ace3ac414eb671b0e87880d683.svg Error
/content/drive/My Drive/dataset/_basic/Poliwag/456c5063598a46a68a60ebcde5647ace.svg Error
completed


In [13]:

from sklearn.model_selection import train_test_split

In [14]:

X = np.array(X).reshape(-1, 96, 96, 3)


X = X / 255.0

y = to_categorical(Y, num_classes = len(labels))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, shuffle = True, random_state = 7)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense

In [18]:
model = Sequential()
model.add(Conv2D(32, 3, padding = 'same', activation = 'relu', input_shape =(96, 96, 3), kernel_initializer = 'he_normal'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal', activation = 'relu'))
model.add(BatchNormalization(axis = -1))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(len(pokemon_types), activation = 'softmax'))

model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 96, 96, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 48, 48, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 64)      

In [19]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

                             

In [20]:
history = model.fit(X_train, y_train, epochs = 20, validation_data=(X_test,y_test))

Epoch 1/20
110/110 [==============================] - 105s 937ms/step - loss: 4.5444 - accuracy: 0.1011 - val_loss: 5.5005 - val_accuracy: 0.0355
Epoch 2/20
110/110 [==============================] - 101s 916ms/step - loss: 3.8186 - accuracy: 0.1570 - val_loss: 4.1905 - val_accuracy: 0.0951
Epoch 3/20
110/110 [==============================] - 104s 943ms/step - loss: 3.3706 - accuracy: 0.2163 - val_loss: 3.7363 - val_accuracy: 0.1684
Epoch 4/20
110/110 [==============================] - 103s 941ms/step - loss: 3.0643 - accuracy: 0.2621 - val_loss: 3.1247 - val_accuracy: 0.2417
Epoch 5/20
110/110 [==============================] - 104s 945ms/step - loss: 2.8320 - accuracy: 0.2885 - val_loss: 2.9412 - val_accuracy: 0.2692
Epoch 6/20
110/110 [==============================] - 102s 926ms/step - loss: 2.7094 - accuracy: 0.3145 - val_loss: 2.7176 - val_accuracy: 0.3207
Epoch 7/20
110/110 [==============================] - 103s 937ms/step - loss: 2.5025 - accuracy: 0.3592 - val_loss: 2.5519 -

In [21]:
model.save("/content/drive/My Drive/pokemon_model/epoch20.h5")